# Introduction

This notebook will train a very simple model that will compare two numbers and tell us if one is larger than the other.

We'll convert the model into something that can be used by tflite and then run it on the ESP32

In [ ]:
import tensorflow as tf

import numpy as np
import os
import pandas as pd
import numpy as np
import sys
import time
import requests
import threading
import tty
import termios
from datetime import datetime, timedelta
now = datetime.now()
import tensorflow as tf
from tensorflow import keras
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from keras import callbacks
from sklearn.model_selection import train_test_split 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import keras
from keras import layers
import itertools
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset

## Create a dataset to train our model
We'll create a python generator and feed that through a tensorflow Dataset to train our model

In [32]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # our input data is an array containing 2 numbers
        X = [number1, number2]
        # our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # our generator should return the input data and the label
        yield X, [Y]
        
# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

## Our very simple mode

We don't need a very complicated model for our problem, so we'll just define a small neural network with an input layer and an output layer.

It's important that the activation function for the output should be sigmoid. This activation function will output a value between 0 and 1.

In [37]:
model = keras.Sequential()
model.add(layers.Input(shape=(2,)))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


## Compile our model¶
For our loss function we need to use BinaryCrossentropy.

Crossentropy quantifies the difference between two probability distribution.

We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.


In [38]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 5)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21 (84.00 B)

 Trainable params: 21 (84.00 B)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.fit(
    train_dataset,
    steps_per_epoch=1000,
    epochs=4
)

Epoch 1/4
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6862 - loss: 0.6204
Epoch 2/4
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9918 - loss: 0.3087
Epoch 3/4
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9967 - loss: 0.1780
Epoch 4/4
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9977 - loss: 0.1298


## Testing our model
We can feed in some values and see what our model predicts

In [43]:
test_X = np.array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.1],
    [0.7, 0.2]
])
Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

[[0.79]
 [0.79]
 [0.01]
 [0.00]]


## Export our model for tflite
We need to convert our model into a tflite model

In [41]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /var/folders/x8/nf9dsl317yl_1wcwtt70k9vh0000gn/T/tmpizfz427o/assets


INFO:tensorflow:Assets written to: /var/folders/x8/nf9dsl317yl_1wcwtt70k9vh0000gn/T/tmpizfz427o/assets


Saved artifact at '/var/folders/x8/nf9dsl317yl_1wcwtt70k9vh0000gn/T/tmpizfz427o'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  5914760416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5925955968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5925957904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5925958608: TensorSpec(shape=(), dtype=tf.resource, name=None)


/Users/mitchelreints/Desktop/05_Development/#Projecten/PEE51 - AIRegelsysteem/.venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1749330359.026538 13564185 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1749330359.026702 13564185 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-07 23:05:59.026909: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/x8/nf9dsl317yl_1wcwtt70k9vh0000gn/T/tmpizfz427o
2025-06-07 23:05:59.027108: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-07 23:05:59.027113: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/x8/nf9dsl317yl_1wcwtt70k9vh0000gn/T/tmpizfz427o
I0000 00:00:1749330359.028937 13564185 mlir_graph_optimization_pass.cc:425] MLIR V1 o

2520

## To convert to C++
We can then run this command to convert the model to c code.
```
xxd -i converted_model.tflite > model_data.cc
```